In [3]:
import pandas as pd 
import numpy as np 
import os


In [26]:
def load_all_json_gz_to_dataframe(directory):
    all_data = pd.DataFrame()
    for filename in os.listdir(directory):
        if filename.endswith(".json.gz"):
            file_path = os.path.join(directory, filename)
            df = pd.read_json(file_path, compression='gzip', lines=True)
            filtered_df = df[(df['selftext'] != '[removed]') & (df['selftext'] != '[deleted]')] # alot of the posts have their data removed
            all_data = pd.concat([all_data, filtered_df], ignore_index=True)
    
    return all_data
directory_path = 'reddit-subset/submissions'
df = load_all_json_gz_to_dataframe(directory_path)
print(df)


       archived                author  created_utc  edited  hide_score  \
0         False            mightymikk   1676535927     0.0       False   
1         False  GiftSpoilerThrowAway   1676536644     0.0       False   
2         False  Entire-Scientist4242   1676538815     0.0       False   
3         False   Senior_Freedom_3291   1676539785     0.0       False   
4         False  unreasonablereason20   1676540110     0.0       False   
...         ...                   ...          ...     ...         ...   
21519     False     East_Manager_5662   1675420344     0.0       False   
21520     False         twinkedgelord   1675421509     NaN       False   
21521     False    thisisnotmedefsnot   1675422401     NaN       False   
21522     False       throwawaaaay212   1675422429     NaN       False   
21523     False             [deleted]   1675423923     NaN       False   

            id  is_self        name  num_comments  over_18  ...  \
0      113lqn7     True  t3_113lqn7         